In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [ ]:
data_main_clean_v4=pd.read_pickle('data_main_clean_v4.pickle')

In [7]:
! pip install transformers

     |████████████████████████████████| 778kB 2.6MB/s 
     |████████████████████████████████| 3.0MB 12.7MB/s 
     |████████████████████████████████| 1.1MB 36.0MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4a97e5e4901f1f42b12d038fa0676f1020afcd9a9d0527904489f1abe68f1063
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## BERT Hugging Face

In [8]:
from transformers import AutoTokenizer, pipeline, TFDistilBertModel
from tqdm import tqdm

In [ ]:
title=data_main_clean_v4['Cleaned_Title'].values

In [9]:
title_list=list(title)

In [10]:
#Using the pretained 'distilbert-base-uncased' model and saving in model
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

#Tokenizing the pretrained model and storing in tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

#Creating a pipeline from the defined model and tokenizer for our data
pipe = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
#Using the defined pipeline to convert our data into features and considering one 768 dimension vecotor for each datapoint
features = pipe(title_list)
features = np.squeeze(features)
features = features[:,0,:]

In [ ]:
#Converting list to numpy array
features1=np.array(features)

In [19]:
features1.shape

(1396270,768)


In [ ]:
a1=features1

In [ ]:
def Recomend(string):
  stopwords_1 = stopwords.words("english")
  a=string
  sent_1=a.lower().strip()
  sent_1 = re.sub(r"won\'t", "will not", sent_1)
  sent_1 = re.sub(r"can\'t", "can not", sent_1)
  sent_1 = re.sub(r"n\'t", " not", sent_1)
  sent_1 = re.sub(r"\'re", " are", sent_1)
  sent_1 = re.sub(r"\'s", " is", sent_1)
  sent_1 = re.sub(r"\'d", " would", sent_1)
  sent_1 = re.sub(r"\'ll", " will", sent_1)
  sent_1 = re.sub(r"\'t", " not", sent_1)
  sent_1 = re.sub(r"\'ve", " have", sent_1)
  sent_1 = re.sub(r"\'m", " am", sent_1)
  sent_1 = re.sub('[^A-Za-z0-9-+]+', ' ', sent_1)
  sent_1 = ' '.join(e for e in sent_1.split() if e not in stopwords_1)

  sent_1=sent_1.lower().strip()
  print('QUERY ENTERED BY THE USER')
  print(sent_1)
  print('\n')

  query=a1[0]
  distance = pairwise_distances(a1, query.reshape(1,-1),metric='cosine')
  indices = np.argsort(distance.flatten())[0:10]
  pdists = np.sort(distance.flatten())[0:10]

  print('RECOMENDED SIMILAR QUESTIONS')
  g=0
  for i in indices:
    g=g+1
    print(g ,'th question','"',data_main_clean_v4['Cleaned_Title'][i],'"')
    print(g ,'th question distance is ',round((float(distance[i])),4))
    print('\n')

In [29]:
import time
start_time = time.time()
Recomend('implementing boundary value analysis software testing c++ program')
print('TIME TAKEN TO FETCH RESULTS')
print(time.time()-start_time,'seconds')

QUERY ENTERED BY THE USER
implementing boundary value analysis software testing c++ program

RECOMENDED SIMILAR QUESTIONS
1 th question ' implementing boundary value analysis software testing c++ program '
1 th question distance is 0.0

2 th question ' implementing data structures algorithms c++ '
2 th question distance is 0.4653

3 th question ' boundary value analysis c++ cppunit '
3 th question distance is 0.472

4 th question ' code metrics analysis unmanaged c++ code '
4 th question distance is 0.4922

5 th question ' c++ code profiling analysis mac mpi '
5 th question distance is 0.5023

6 th question ' calculating critical path dag c++ '
6 th question distance is 0.5064

7 th question ' methods implementing using graphs nodes c++ '
7 th question distance is 0.5136

8 th question ' complex data structures embedding extending python c++'
8 th question distance is 0.5187

9 th question ' obfuscate c++ variables functions '
9 th question distance is 0.5244

10 th question ' c++ stat